In [ ]:
import csv
import numpy as np
import Loader
import tensorflow as tf
from sklearn import preprocessing
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.losses import MeanSquaredError

In [ ]:
compoundsTrain, smilesTrain, labelsTrain, compoundDataTrain, activitiesTrain = Loader.getTrain(defaultValue=0)
compoundsTest, smilesTest, labelsTest, compoundDataTest, activitiesTest = Loader.getTest(defaultValue=0)
compoundsValidate, smilesValidate, labelsValidate, compoundDataValidate, activitiesValidate = Loader.getValidate(defaultValue=0)

#print(labelsTrain)
#print(compoundsTrain)
#print(smilesTrain)
#print(activitiesTrain)

#for i in range(len(labelsTrain)):
#    print(labelsTrain[i] + ": ", compoundDataTrain[0,i])

def toClassification(y): # The resulting array will contain values of -1 if it is below 4.5 and 1 if it is above
    y = np.array(y)
    classification = (y.astype(float)>4).astype(int)
    return classification * 2 - 1

def normalizeData(train,test,validate):
    for i in range(np.shape(train)[1]):
        std = np.std(train[:,i])
        mean = np.mean(train[:,i])
        if(std == 0):
            std = 1
        train[:,i] = (train[:,i] - mean) / std
        test[:,i] = (test[:,i] - mean) / std
        validate[:,i] = (validate[:,i] - mean) / std
    return train, test, validate

def normalize_2d(matrix):
    norm = np.linalg.norm(matrix,np.inf)
    matrix = matrix/norm  # normalized matrix
    return matrix

normalizeData(compoundDataTrain, compoundDataTest, compoundDataValidate)

print(len(compoundsTrain), len(compoundsTest), len(compoundsValidate))

In [ ]:
model = Sequential()
model.add(Dense(250, input_dim=np.shape(compoundDataTrain)[1], activation='softmax', kernel_regularizer = keras.regularizers.L2(0.001)))
model.add(Dense(150, activation='tanh', kernel_regularizer = keras.regularizers.L1(0.001)))
model.add(Dense(75, activation='relu', kernel_regularizer = keras.regularizers.L1(0.001)))
model.add(Dense(150, activation='relu', kernel_regularizer = keras.regularizers.L1(0.001)))
model.add(Dense(100, activation='tanh', kernel_regularizer = keras.regularizers.L1(0.001)))
model.add(Dense(1, activation='tanh', kernel_regularizer = keras.regularizers.L1(0.001)))

model.compile(loss='MeanSquaredError', optimizer='adam', metrics=['accuracy'])
history = model.fit(compoundDataTrain, toClassification(activitiesTrain),validation_data = 
                    (compoundDataTest, toClassification(activitiesTest)),epochs=100, batch_size=32)